In [28]:
# class ComputeGraph:
#     def __init__(self):
#         self.graph = []


# https://en.wikipedia.org/wiki/Genetic_programming#Program_representation

class Tree:
    def __init__(self, left_tree = None, right_tree = None, comment = None):
        self.left_tree = left_tree
        self.right_tree = right_tree
        
    def __add__(self, obj):
        if type(obj) == int or type(obj) == float:
            return Operation("+", self, Scalar(obj))
        else:
            return Operation("+", self, obj)
        
    def __sub__(self, obj):
        if type(obj) == int or type(obj) == float:
            return Operation("-", self, Scalar(obj))
        else:
            return Operation("-", self, obj)
        
    def __truediv__(self, obj): 
        if type(obj) == int or type(obj) == float:
            return Operation("/", self, Scalar(obj))
        else:
            return Operation("/", self, obj)
        
    def __neg__(self):
        return Operation("-", Scalar(0), self)
        
    
class Operation(Tree):
    def __init__(self, op, left_tree = None, right_tree = None):
        self.op = op
        super().__init__(left_tree, right_tree, None)
        
    def __repr__(self):
        return f"({self.op} {self.left_tree.__repr__()} {self.right_tree.__repr__()})"
        
class Scalar(Tree):
    def __init__(self, scalar):
        self.scalar = scalar
        super().__init__(None, None, None)
        
    def __repr__(self):
        return str(self.scalar)

class Reference(Tree):
    """A reference to a flow, stock, or variable."""
    def __init__(self, name):
        self.name = name
        super().__init__(None, None, None)
        
    def __repr__(self):
        return f"\"{self.name}\""
        
        
        
        
class Stock(Reference):
    """Implicit 'dt', must always keep backup of last val"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def __add__(self, obj):
        pass
        
    def __div__(self, obj):
        pass
        

        
class Flow(Reference):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        


In [34]:
-Scalar(4)

(- 0 4)

In [35]:
Scalar(5) + Scalar(4) / Reference("groundwater")

(+ 5 (/ 4 "groundwater"))

In [36]:
(Scalar(5) + Scalar(4)) / Reference("groundwater")

(/ (+ 5 4) "groundwater")

In [ ]:
class Model:
    def __init__(self):
        self.things = {}
        
    # actually, as long as operations return a new thing, this should just work.
    def __setattr__(self, name, value):
        if name not in self.things:
            self.things[name] = value
        else:
            self.things[name] = self.things[name] + value
            
    def _compile(self):
        """Write optimized jax operation list for computations into python file?"""
        pass

In [ ]:





m = Model()

m.clouds = Stock()
m.groundwater = Stock()
m.food = Stock()

m.food = (m.clouds + 3) * m.groundwater # an implicit flow?





In [ ]:
# how do we model moving values from one stock to another?


# involves one stock starting with an initial value, a flow, and another stock.

m = Model()
m.from_stock = Stock()
m.to_stock = Stock()
m.movement_flow = Flow()


#m.flow(from_stock, to_stock, m.from_stock / 2)
#m.to_stock <= m.from_stock / 2 # I think beyond simple cases it's too hard to imply what it's actually being removed "from"

# a flow implies things leaving one stock and entering another.


# way too verbose
m.to_stock[m.t+1] = m.to_stock[m.t] + m.from_stock[m.t] / 2
m.from_stock[m.t+1] = m.to_stock[m.t] + m.from_stock[m.t] / 2

# more obvious directionality, but dislike
m.movement_flow <= m.from_stock / 2
m.to_stock <= m.movement_flow

# from https://en.wikipedia.org/wiki/Stock_and_flow
# "stock" = "level"
# "flow" = "rate"
# I prefer their "order of execution" translation formulas.
# -----------------------------------
m.movement_flow = m.from_stock / 2
m.from_stock -= m.movement_flow
m.to_stock += m.movement_flow
# -----------------------------------
# or (technically the more integral-y form, an implicit 'dt')
# -----------------------------------
m.movement_flow = m.from_stock / 2    << "this is a test"                               # two nodes: "from_stock" "/ 2"
m.from_stock = -m.movement_flow       << "We move out of from stock"
m.to_stock = m.movement_flow          << "And into the to stock"
# -----------------------------------
# then we can set initial conditions by overriding __getitem__:
m.from_stock[0] = 10 # if we want to hold it static?


